In [1]:
import requests
import json
from selenium import webdriver
import pandas as pd
from lxml import etree
import time

In [279]:
def get_districts(url, y):
    driver.get(url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)
    district_links = html.xpath("/html/body/div[@class='main-content']/div[@class='div-border items-list']/div[@class='items']/span[@class='elem-l']/a/@href")
    district_names = [link[link.index(y)+5:-1] for link in district_links]
    temp_districts = {name:link for name,link in zip(district_names, district_links)}
    temp_districts['whole_city'] = url
    return temp_districts


def get_data(url, city, name):
    price_city = pd.DataFrame(columns=['date', 'city_district', 'price', 'increase'])
    driver.get(url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)
    len_month_up = len(html.xpath("/html/body/div[@class='main-content']/div[@class='avger clearfix']\
                    /div[@class='fjlist-wrap clearfix']/div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix up']"))
    len_month_down = len(html.xpath("/html/body/div[@class='main-content']/div[@class='avger clearfix']\
                    /div[@class='fjlist-wrap clearfix']/div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix down']"))
    len_month_no_change = len(html.xpath("/html/body/div[@class='main-content']/div[@class='avger clearfix']\
                    /div[@class='fjlist-wrap clearfix']/div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix nochange']"))
    len_all = {'up':len_month_up, 'down':len_month_down, 'nochange':len_month_no_change}
    for i in len_all.keys():
        if len_all[i] > 0:
            for mon in range(1, len_all[i]+1):
                y_m = html.xpath(f"/html/body/div[@class='main-content']/div[@class='avger clearfix']/div[@class='fjlist-wrap clearfix']\
                     /div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix {i}'][{mon}]/a[@class='nostyle']/b")
                y_m = y_m[0].xpath('string()')
                y_m = y_m[:4] + '-' + y_m[5:7]
                price = html.xpath(f"/html/body/div[@class='main-content']/div[@class='avger clearfix']/div[@class='fjlist-wrap clearfix']\
                     /div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix {i}'][{mon}]/a[@class='nostyle']/span")
                try:
                    price = int(price[0].xpath('string()').split('元')[0])
                except:
                    price = None
                increase = html.xpath(f"/html/body/div[@class='main-content']/div[@class='avger clearfix']/div[@class='fjlist-wrap clearfix']\
                     /div[@class='fjlist-box boxstyle2'][1]/ul/li[@class='clearfix {i}'][{mon}]/a[@class='nostyle']/em")
                if i != 'nochange':
                    increase = round(float(increase[0].xpath('string()').split('%')[0])/100, 4)
                else:
                    increase = None
                temp = dict()
                temp['date'] = y_m
                temp['city_district'] = city + '_' + name
                temp['price'] = price
                if i == 'down':
                    temp['increase'] = -increase
                else:
                    temp['increase'] = increase
                price_city = price_city.append(temp, ignore_index=True)
    return price_city
        
def work(start, end, cities):
    range_y = [str(year) for year in range(start, end+1)]
    price_city = pd.DataFrame(columns=['date', 'city_district', 'price', 'increase'])
    for y in range_y:
        for city in cities:
            url = f'https://www.anjuke.com/fangjia/{city + y}/'
            temp_districts = get_districts(url, y)        
            for name in temp_districts.keys():
                temp = get_data(temp_districts[name], city, name)
                price_city = price_city.append(temp)
            time.sleep(1)
    return price_city

In [282]:
driver = webdriver.Chrome(executable_path='/Users/paulhwang/Desktop/chromedriver')
price_city = work(2011, 2020, ['shenzhen', 'guangzhou', 'hangzhou', 'shanghai', 'beijing'])
price_city.to_csv('price_cites.csv', index=False)